In [1]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.optimize import minimize
from sklearn.metrics import log_loss

In [2]:
def predict(logits):
    _, preds = torch.max(logits.data, 1)
    return preds

In [3]:
def predict_proba(logits):
    div = torch.sum(torch.exp(logits), dim=1)[:, None]
    probs = torch.exp(logits) / div
    return probs

## Усреднение вероятностей

In [4]:
logits = torch.load("../cv/output_test_1.pt")

In [5]:
div = torch.sum(torch.exp(logits), dim=1)[:, None]
probs = torch.exp(logits) / div

In [6]:
all_probs = []
for i in range(5):
    logits = torch.load(f"../cv/output_test_{i + 1}.pt")
    all_probs.append(predict_proba(logits))

In [7]:
mean_pred = torch.stack(all_probs).mean(dim=0)

In [8]:
_, predicted = torch.max(mean_pred, 1)

In [9]:
df = pd.read_csv('../../data/intro-dl-2024/sample_submission.csv')
df['label'] = predicted
df.to_csv("mean_sub.csv", index=False)

## Усреднение логитов (странно но пусть будет)

In [16]:
all_logits = []
for i in range(5):
    logits = torch.load(f"../cv/output_test_{i + 1}.pt")
    all_logits.append(logits)

In [17]:
mean_pred = predict_proba(torch.stack(all_logits).mean(dim=0))

In [18]:
_, predicted = torch.max(mean_pred, 1)

In [19]:
df = pd.read_csv('../../data/intro-dl-2024/sample_submission.csv')
df['label'] = predicted
df.to_csv("mean_logits_sub.csv", index=False)

## Linear maximization

In [36]:
exm = torch.load("../cv/output_train_1.pt")
df = pd.read_csv('../../data/intro-dl-2024/train.csv')

In [37]:
len(df), exm.shape

(8144, torch.Size([8144, 196]))

In [38]:
train_idx, val_idx, y_train, y_val = train_test_split(range(exm.shape[0]), df['label'], train_size=0.9)

In [39]:
def train_step(w):
    result = torch.zeros_like(exm[train_idx])
    w = [element / w.sum() for element in w]
    for i in range(5):
        logits = torch.load(f"../cv/output_train_{i + 1}.pt")
        preds = predict_proba(logits)
        result += preds[train_idx] * w[i]
    return -log_loss(y_true=y_train, y_pred=result)

In [43]:
result = minimize(train_step, x0=(0, 0, 0.2, 0.2, 0.2), bounds=[(0, 1), (0, 1), (0, 1), (0, 1), (0, 1)])

/Users/rozalina/DL/DL_6sem/contest/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:2981: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
  warnings.warn(
/Users/rozalina/DL/DL_6sem/contest/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:2981: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
  warnings.warn(
/Users/rozalina/DL/DL_6sem/contest/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:2981: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
  warnings.warn(
/Users/rozalina/DL/DL_6sem/contest/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:2981: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
  warnings.warn(
/Users/rozalina/DL/DL_6sem/contest/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:

In [45]:
result.x

array([4.78804263e-09, 4.41409335e-09, 2.00000002e-01, 2.00000002e-01,
       2.00000002e-01])

### Не получилось оптимизировать :(

## Голосование

In [74]:
all_preds= []
for i in range(5):
    logits = torch.load(f"../cv/output_test_{i + 1}.pt")
    all_preds.append(predict(logits))

In [75]:
all_preds = torch.stack(all_preds).T

In [76]:
vote_preds = [int(max(set(row), key = list(row).count)) for row in all_preds]

In [77]:
df = pd.read_csv('../../data/intro-dl-2024/sample_submission.csv')
df['label'] = vote_preds
df.to_csv("vote_sub.csv", index=False)